In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0.0)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 {ability} 에 능숙한 어시스턴트입니다. 20자 이내로 응답하세요",
        ),
        # 대화 기록을 변수로 사용, history 가 MessageHistory 의 key 가 됨
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # 사용자 입력을 변수로 사용
    ]
)
runnable = prompt | model  # 프롬프트와 모델을 연결하여 runnable 객체 생성


In [40]:
def summarize_retrieved_documents(filtered_results, query):
    """검색된 문서를 LLM을 활용하여 요약"""
    if not filtered_results:
        return ""

    document_texts = "\n\n".join([
        f"[유사도: {score:.2f}]\n{doc.page_content}" 
        for doc, score in filtered_results
    ])

    print(f'#################################################\ndocument_texts: \n{document_texts}')

    # LLM을 사용하여 문서 요약
    prompt = f"""
    다음은 이전 대화 내역에서 현재 질문 "{query}"와 관련성이 높은 부분들입니다. 이를 참고하여 다음 지침에 따라 요약해주세요:

    1. 현재 질문에 직접적으로 관련된 정보를 우선적으로 추출하세요.
    2. 코드 블록과 그 설명은 온전히 보존하세요.
    3. 유사도 점수가 높은 내용에 더 큰 가중치를 두세요.
    4. 정보를 다음 형식으로 구조화하세요:
    - 핵심 개념/용어 설명
    - 관련 코드 예시
    - 주요 인사이트/팁
    5. 기술적 정확성을 유지하면서 중복 정보는 제거하세요.
    6. 최신 대화 내용을 더 관련성 높게 처리하세요.

    {document_texts}
    """
    summarized_result = model.invoke(prompt)
    return summarized_result.content.strip()


In [41]:
from utils.vector_handler import initialize_vector_store

def search_similar_questions(internal_id, query, top_k=5, similarity_threshold=0.7):
    """벡터DB에서 사용자의 질문과 유사한 질문 검색"""
    vectorstore = initialize_vector_store(internal_id)  # 세션별 벡터스토어 로드
    
    # 🔎 유사도 점수와 함께 검색 실행
    search_results = vectorstore.similarity_search_with_score(query, k=top_k*2)  # 더 많은 결과를 가져와서 필터링
    
    # 유사도 점수가 threshold를 넘는 결과만 필터링
    filtered_results = []
    seen_content = set()  # 중복 콘텐츠 확인용 집합
    
    for doc, score in search_results:
        # FAISS의 score는 L2 거리이므로 코사인 유사도로 변환 (1 - score/2가 코사인 유사도의 근사값)
        cosine_sim = 1 - (score / 2)
        
        if cosine_sim >= similarity_threshold:
            # 콘텐츠 핵심 부분 추출 (사용자 질문 부분만)
            content_key = ""
            for line in doc.page_content.split('\n'):
                if "사용자 질문:" in line:
                    content_key = line.strip()
                    break
            
            # 중복 콘텐츠 건너뛰기
            if content_key and content_key in seen_content:
                continue
            
            # 가중치 계산 (콘텐츠 품질 기반)
            weight = 1.0
            if "validated_code: None" in doc.page_content or "코드 없음" in doc.page_content:
                weight *= 0.8  # 코드가 없는 경우 가중치 감소
            
            if "인사이트: None" in doc.page_content or "인사이트 없음" in doc.page_content:
                weight *= 0.9  # 인사이트가 없는 경우 가중치 감소
                
            if "실행된 코드:" in doc.page_content and "코드 없음" not in doc.page_content:
                weight *= 1.3  # 실행된 코드가 있는 경우 가중치 증가
                
            if "생성된 인사이트:" in doc.page_content and "인사이트 없음" not in doc.page_content:
                weight *= 1.2  # 인사이트가 있는 경우 가중치 증가
            
            # 최종 스코어 조정
            adjusted_score = cosine_sim * weight
            
            if content_key:
                seen_content.add(content_key)
            
            filtered_results.append((doc, adjusted_score))
    
    # 조정된 점수로 상위 결과 선택
    filtered_results.sort(key=lambda x: x[1], reverse=True)
    filtered_results = filtered_results[:3]  # 상위 top_k개만 유지
    
    # 결과가 있는 경우에만 컨텍스트 생성
    if filtered_results:
        retrieved_context = "\n\n".join([
            f"[유사도: {score:.2f}]\n{doc.page_content}" 
            for doc, score in filtered_results
        ])
    else:
        retrieved_context = ""
    retrieved_context = summarize_retrieved_documents(filtered_results, query)
    
    return retrieved_context

In [42]:
internal_id = "temp_KSW_20250225_1118"
query = "그래프 해석 가능할까요?"
retrieved_context = search_similar_questions(internal_id, query)

🔢 [initialize_vector_store] 벡터DB 로드 시작 (세션: temp_KSW_20250225_1118)
#################################################
document_texts: 
[유사도: 1.19]

            사용자 질문: 기준년월에 따른  CMIP의 추세를 알고 싶습니다.
            AI 응답: 분석이 완료되었습니다! 아래 결과를 확인해주세요.
            실행된 코드: ```python
import pandas as pd

# 기준년월별 변액종신CMIP의 평균 추세 계산
cmip_trend = df.groupby('기준년월')['변액종신CMIP'].mean().round(2)

# 결과 저장
analytic_results = {
    'CMIP_Trend': cmip_trend
}

# 집계성 데이터 출력
print(cmip_trend)
```
            분석 결과: {'CMIP_Trend': 기준년월
202405    17327.30
202406    17320.91
202407    17322.96
202408    17323.08
202409    17327.54
202410    17315.90
Name: 변액종신CMIP, dtype: float64}
            생성된 인사이트: 1. 주요 발견사항
   - CMIP(변액종신CMIP)의 추세를 분석한 결과, 2024년 5월부터 2024년 10월까지의 데이터에서 큰 변동 없이 비교적 안정적인 추세를 보이고 있습니다. CMIP 값은 17315.90에서 17327.54 사이에서 움직이고 있으며, 월별로 큰 변화는 관찰되지 않았습니다.

2. 특이점
   - 2024년 10월에 CMIP 값이 약간 감소한 17315.90을 기록하였으나, 이는 전체적인 추세에 큰 영향을 미치지 않는 수준입니다. 전반적으로 CMIP 값은 안정적인 수준을 유지하고 있습니다.

3. 추천 사항
   - CMIP의 

In [43]:
print(retrieved_context)

### 핵심 개념/용어 설명
- **그래프 해석**: 그래프 해석은 데이터를 시각적으로 표현하여 데이터 간의 관계나 패턴을 파악하는 과정입니다. 이를 통해 복잡한 데이터 세트를 더 쉽게 이해할 수 있습니다.
- **그래프 컴파일**: 그래프 컴파일은 데이터를 그래프로 변환하여 시각적으로 표현하는 과정입니다. Python에서는 Matplotlib, Seaborn, Plotly와 같은 라이브러리를 사용하여 그래프를 생성할 수 있습니다.

### 관련 코드 예시
```python
import pandas as pd

# 기준년월별 변액종신CMIP의 평균 추세 계산
cmip_trend = df.groupby('기준년월')['변액종신CMIP'].mean().round(2)

# 결과 저장
analytic_results = {
    'CMIP_Trend': cmip_trend
}

# 집계성 데이터 출력
print(cmip_trend)
```

### 주요 인사이트/팁
1. **데이터 준비**: 그래프에 사용할 데이터를 정리하고 필요한 형식으로 변환합니다.
2. **그래프 유형 선택**: 데이터의 특성과 분석 목적에 맞는 그래프 유형(예: 막대 그래프, 선 그래프, 산점도 등)을 선택합니다.
3. **레이블 및 제목 추가**: 그래프의 축, 제목, 범례 등을 추가하여 그래프의 의미를 명확히 합니다.
4. **스타일링**: 그래프의 색상, 폰트, 크기 등을 조정하여 가독성을 높입니다.

### 추가 인사이트
- CMIP(변액종신CMIP)의 추세를 분석한 결과, 2024년 5월부터 2024년 10월까지의 데이터에서 큰 변동 없이 비교적 안정적인 추세를 보이고 있습니다. CMIP 값은 17315.90에서 17327.54 사이에서 움직이고 있으며, 월별로 큰 변화는 관찰되지 않았습니다.
- CMIP의 안정적인 추세를 유지하기 위해 지속적인 모니터링과 외부 경제 환경 변화에 대한 민감도 분석이 필요합니다.


ValueError: not enough values to unpack (expected 2, got 1)